In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
forest_trans = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Combined_CoverType_original_data2.csv')

In [0]:
forest_trans = forest_trans.drop(columns = forest_trans.columns[0:1])

In [0]:
forest_trans['Wilderness'] = forest_trans['Wilderness'].astype('category')
forest_trans['Soil'] = forest_trans['Soil'].astype('category')
forest_trans['CoverType'] = forest_trans['CoverType'].astype('category')

In [0]:
forest_trans.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness,Soil,CoverType
0,2596,51,3,258,0,510,221,232,148,6279,1,29,4
1,2590,56,2,212,-6,390,220,235,151,6225,1,29,4
2,2804,139,9,268,65,3180,234,238,135,6121,1,12,2
3,2785,155,18,242,118,3090,238,238,122,6211,1,30,2
4,2595,45,2,153,-1,391,220,234,150,6172,1,29,4


# Transforming Original Data

In [0]:
forest_trans['Vertical_Scaled'] = forest_trans['Vertical_Distance_To_Hydrology'] - min(forest_trans['Vertical_Distance_To_Hydrology'])

In [0]:
forest_trans.drop(columns = ['Vertical_Distance_To_Hydrology'], inplace = True)

In [0]:
forest_trans['Elevation'] = forest_trans['Elevation']
forest_trans['Aspect'] = forest_trans['Aspect']
forest_trans['Slope'] = np.cos(forest_trans['Slope'])
forest_trans['Horizontal_Distance_To_Hydrology'] = np.sqrt(forest_trans['Horizontal_Distance_To_Hydrology'])
forest_trans['Vertical_Scaled'] = np.log(forest_trans['Vertical_Scaled']+1)
forest_trans['Horizontal_Distance_To_Roadways'] = np.sqrt(forest_trans['Horizontal_Distance_To_Roadways'])
forest_trans['Hillshade_9am'] = forest_trans['Hillshade_9am']
forest_trans['Hillshade_Noon'] = forest_trans['Hillshade_Noon']
forest_trans['Hillshade_3pm'] = forest_trans['Hillshade_3pm']
forest_trans['Horizontal_Distance_To_Fire_Points'] = np.sqrt(forest_trans['Horizontal_Distance_To_Fire_Points'])
forest_trans['Wilderness'] = forest_trans['Wilderness'].astype('category')
forest_trans['Soil'] = forest_trans['Soil'].astype('category')
forest_trans['CoverType'] = forest_trans['CoverType'].astype('category')

Saving the Dataframe for future refernce

In [0]:
forest_trans.to_csv('/content/drive/My Drive/Colab Notebooks/Combined_CoverType_Original_Trans.csv')

In [0]:
forest_trans = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Combined_CoverType_Original_Trans.csv')

In [0]:
forest_trans = forest_trans.drop(columns = forest_trans.columns[0:1])

In [0]:
forest_trans['Wilderness'] = forest_trans['Wilderness'].astype('category')
forest_trans['Soil'] = forest_trans['Soil'].astype('category')
forest_trans['CoverType'] = forest_trans['CoverType'].astype('category')

In [6]:
forest_trans.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness,Soil,CoverType,Vertical_Scaled
0,2596,51,-0.989992,16.062378,22.583180,221,232,148,79.240141,1,29,4,5.159055
1,2590,56,-0.416147,14.560220,19.748418,220,235,151,78.898669,1,29,4,5.123964
2,2804,139,-0.911130,16.370706,56.391489,234,238,135,78.236820,1,12,2,5.476464
3,2785,155,0.660317,15.556349,55.587768,238,238,122,78.809898,1,30,2,5.676754
4,2595,45,-0.416147,12.369317,19.773720,220,234,150,78.562077,1,29,4,5.153292


# Modeling

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

Decision Tree

In [0]:
x = forest_trans.drop(columns= ['CoverType'])
y = forest_trans['CoverType']

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .3, random_state=25)

In [0]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV 

In [0]:
dt = DecisionTreeClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
dt.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=19, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=25, splitter='best')

In [0]:
y_testpred = dt.predict(x_test)

In [0]:
print('Training Accuracy for DecisionTree Classifier: ', accuracy_score(y_train,dt.predict(x_train)))

Training Accuracy for DecisionTree Classifier:  0.9236405480098744


In [0]:
print('Training Accuracy for DecisionTree Classifier: ', accuracy_score(y_test,y_testpred))

Training Accuracy for DecisionTree Classifier:  0.8975123921424637


In [0]:
print(classification_report(y_test, y_testpred))

              precision    recall  f1-score   support

           1       0.90      0.88      0.89     63514
           2       0.90      0.92      0.91     85045
           3       0.90      0.90      0.90     10747
           4       0.88      0.84      0.86     14998

    accuracy                           0.90    174304
   macro avg       0.89      0.89      0.89    174304
weighted avg       0.90      0.90      0.90    174304



In [0]:
print(confusion_matrix(y_test,y_testpred))

[[55905  7184     4   421]
 [ 5810 78283   306   646]
 [    5   341  9721   680]
 [  561  1194   712 12531]]


In [0]:
pd.DataFrame({'columns':x.columns,'Importance':dt.feature_importances_})

,columns,Importance
0,Elevation,0.238869
1,Aspect,0.017954
2,Slope,0.001014
3,Horizontal_Distance_To_Hydrology,0.050796
4,Horizontal_Distance_To_Roadways,0.127003
5,Hillshade_9am,0.026011
6,Hillshade_Noon,0.026302
7,Hillshade_3pm,0.015316
8,Horizontal_Distance_To_Fire_Points,0.130464
9,Wilderness,0.032572


Random Forest

In [0]:
from sklearn.ensemble import RandomForestClassifier

In [0]:
# model with optimal hyperparameters
rforest = RandomForestClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
rforest.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=19, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=25, verbose=0,
                       warm_start=False)

In [0]:
y_testpred = rforest.predict(x_test)

In [0]:
print('Training Accuracy for Random Forest Classifier: ', accuracy_score(y_train,rforest.predict(x_train)))

Training Accuracy for Random Forest Classifier:  0.94846179568634


In [0]:
print('Training Accuracy for Random Forest Classifier: ', accuracy_score(y_test,y_testpred))

Training Accuracy for Random Forest Classifier:  0.9256815678355058


In [0]:
print(classification_report(y_test, y_testpred))

              precision    recall  f1-score   support

           1       0.94      0.90      0.92     63514
           2       0.91      0.96      0.94     85045
           3       0.93      0.94      0.93     10747
           4       0.94      0.84      0.89     14998

    accuracy                           0.93    174304
   macro avg       0.93      0.91      0.92    174304
weighted avg       0.93      0.93      0.93    174304



In [0]:
print(confusion_matrix(y_test,y_testpred))

[[57257  6079     0   178]
 [ 3170 81390   205   280]
 [    3   280 10102   362]
 [  581  1227   589 12601]]


In [0]:
pd.DataFrame({'columns':x.columns,'Importance':rforest.feature_importances_})

,columns,Importance
0,Elevation,0.273754
1,Aspect,0.030818
2,Slope,0.003607
3,Horizontal_Distance_To_Hydrology,0.045606
4,Horizontal_Distance_To_Roadways,0.112457
5,Hillshade_9am,0.027959
6,Hillshade_Noon,0.030154
7,Hillshade_3pm,0.025703
8,Horizontal_Distance_To_Fire_Points,0.104538
9,Wilderness,0.080708


KNN

In [0]:
from sklearn.preprocessing import StandardScaler

x1 = forest_trans.drop(columns= ['CoverType','Soil','Wilderness'])
y = forest_trans['CoverType']

x_std = StandardScaler().fit_transform(x1)

x_std = pd.DataFrame(x_std, columns = x1.columns)

x = pd.concat([x_std, forest_trans[['Soil','Wilderness']]], axis = 1)
x.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Vertical_Scaled,Soil,Wilderness
0,-1.297805,-0.935157,-1.389008,0.164973,-1.395274,0.330743,0.439143,0.142960,2.587999,-0.867052,29,1
1,-1.319235,-0.890480,-0.578038,-0.056053,-1.567204,0.293388,0.590899,0.221342,2.564187,-1.014305,29,1
2,-0.554907,-0.148836,-1.277558,0.210340,0.655217,0.816364,0.742654,-0.196691,2.518036,0.464875,12,1
3,-0.622768,-0.005869,0.943243,0.090516,0.606471,0.965786,0.742654,-0.536343,2.557997,1.305345,30,1
4,-1.301377,-0.988770,-0.578038,-0.378421,-1.565670,0.293388,0.540313,0.195215,2.540716,-0.891238,29,1


In [0]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .3, random_state=25)

In [0]:
from sklearn.neighbors import KNeighborsClassifier

In [0]:
knn = KNeighborsClassifier(metric = 'cityblock', n_neighbors= 4, weights= 'distance')
knn.fit(x_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='cityblock',
                     metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                     weights='distance')

In [0]:
y_testpred = knn.predict(x_test)

In [0]:
print('Training Accuracy for KNN Classifier: ', accuracy_score(y_train,knn.predict(x_train)))

Training Accuracy for KNN Classifier:  1.0


In [0]:
print('Training Accuracy for KNN Classifier: ', accuracy_score(y_test,y_testpred))

Training Accuracy for KNN Classifier:  0.9185962456398017


In [0]:
print(classification_report(y_test, y_testpred))

              precision    recall  f1-score   support

           1       0.92      0.91      0.92     63514
           2       0.92      0.94      0.93     85045
           3       0.91      0.91      0.91     10747
           4       0.87      0.86      0.87     14998

    accuracy                           0.92    174304
   macro avg       0.91      0.90      0.90    174304
weighted avg       0.92      0.92      0.92    174304



In [0]:
print(confusion_matrix(y_test,y_testpred))

[[57831  5264     6   413]
 [ 4326 79685   304   730]
 [   10   275  9758   704]
 [  537   929   691 12841]]


## Ensemble

In [0]:
#base_LR = LogisticRegression(random_state = 25)
base_DT = DecisionTreeClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
#base_NB = GaussianNB()
base_RF = RandomForestClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
base_KNN = KNeighborsClassifier(metric = 'cityblock', 
                                n_neighbors= 4, 
                                weights= 'distance')

In [0]:
x = forest_trans.drop(columns= ['CoverType'])
y = forest_trans['CoverType']

Bagging Classifier

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .3, random_state=25)

In [0]:
from sklearn.ensemble import BaggingClassifier

Bagged DT

In [0]:
bgcl_DT = BaggingClassifier(base_estimator=base_DT, n_estimators = 51, oob_score=True, random_state=25)

bgcl_DT.fit(x_train, y_train)



BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=19,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=10,
                                                        min_samples_split=10,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=25,

In [0]:
y_testpred = bgcl_DT.predict(x_test)

In [0]:
print('Training Accuracy for Bagged DT: ', accuracy_score(y_train,bgcl_DT.predict(x_train)))

Training Accuracy for Bagged DT:  0.9629660591874268


In [0]:
print('Training Accuracy for Bagged DT: ', accuracy_score(y_test,y_testpred))

Training Accuracy for Bagged DT:  0.9408562052505967


In [0]:
print('OOB Score for Bagged DT:', bgcl_DT.oob_score_)

OOB Score for Bagged DT: 0.9374416042959568


In [0]:
print(classification_report(y_test, y_testpred))

              precision    recall  f1-score   support

           1       0.95      0.93      0.94     63514
           2       0.94      0.96      0.95     85045
           3       0.94      0.95      0.94     10747
           4       0.94      0.88      0.91     14998

    accuracy                           0.94    174304
   macro avg       0.94      0.93      0.93    174304
weighted avg       0.94      0.94      0.94    174304



In [0]:
print(confusion_matrix(y_test,y_testpred))

[[58900  4407     1   206]
 [ 2823 81723   196   303]
 [    4   200 10194   349]
 [  450   873   497 13178]]


Bagged RF

In [0]:
bgcl_RF = BaggingClassifier(base_estimator=base_RF, n_estimators = 51, oob_score=True, random_state=25)
bgcl_RF.fit(x_train, y_train)

BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=True,
                                                        ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=19,
                                                        max_features='auto',
                                                        max_leaf_nodes=None,
                                                        max_samples=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=10,
                                                        min_samples_split=10,
                                                        min_weight_fraction_leaf=0.0,
  

In [0]:
y_testpred = bgcl_RF.predict(x_test)

In [0]:
print('Training Accuracy for Bagged RF: ', accuracy_score(y_train,bgcl_RF.predict(x_train)))

Training Accuracy for Bagged RF:  0.9314397553035593


In [0]:
print('Training Accuracy for Bagged RF: ', accuracy_score(y_test,y_testpred))

Training Accuracy for Bagged RF:  0.9128648797503213


In [0]:
print('OOB Score for Bagged RF:', bgcl_RF.oob_score_)

OOB Score for Bagged RF: 0.9120081237644698


In [0]:
print(classification_report(y_test, y_testpred))

              precision    recall  f1-score   support

           1       0.93      0.89      0.91     63514
           2       0.90      0.95      0.92     85045
           3       0.91      0.93      0.92     10747
           4       0.93      0.80      0.86     14998

    accuracy                           0.91    174304
   macro avg       0.92      0.89      0.90    174304
weighted avg       0.91      0.91      0.91    174304



In [0]:
print(confusion_matrix(y_test,y_testpred))

[[56417  6887     1   209]
 [ 3791 80686   267   301]
 [    3   355 10003   386]
 [  742  1522   724 12010]]


Bagged KNN

In [0]:
bgcl_KNN = BaggingClassifier(base_estimator=base_KNN, n_estimators = 51, oob_score=True, random_state=25)
bgcl_KNN.fit(x_train, y_train)

BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto',
                                                      leaf_size=30,
                                                      metric='cityblock',
                                                      metric_params=None,
                                                      n_jobs=None,
                                                      n_neighbors=4, p=2,
                                                      weights='distance'),
                  bootstrap=True, bootstrap_features=False, max_features=1.0,
                  max_samples=1.0, n_estimators=51, n_jobs=None, oob_score=True,
                  random_state=25, verbose=0, warm_start=False)

In [0]:
y_testpred = bgcl_KNN.predict(x_test)

In [0]:
print('Training Accuracy for Bagged KNN: ', accuracy_score(y_train,bgcl_KNN.predict(x_train)))

Training Accuracy for Bagged KNN:  0.999751664584911


In [0]:
print('Training Accuracy for Bagged KNN: ', accuracy_score(y_test,y_testpred))

Training Accuracy for Bagged KNN:  0.9018898017257205


In [0]:
print('OOB Score for Bagged KNN:', bgcl_KNN.oob_score_)

OOB Score for Bagged KNN: 0.8998052656943065


In [0]:
print(classification_report(y_test, y_testpred))

              precision    recall  f1-score   support

           1       0.91      0.89      0.90     63514
           2       0.91      0.93      0.92     85045
           3       0.87      0.88      0.88     10747
           4       0.86      0.81      0.83     14998

    accuracy                           0.90    174304
   macro avg       0.89      0.88      0.88    174304
weighted avg       0.90      0.90      0.90    174304



In [0]:
print(confusion_matrix(y_test,y_testpred))

[[56562  6536     5   411]
 [ 4923 79044   365   713]
 [    7   407  9451   882]
 [  680  1175   997 12146]]


AdaBoost

In [0]:
from sklearn.ensemble import AdaBoostClassifier

AdaBoost DT

In [0]:
abcl_DT = AdaBoostClassifier(base_estimator=base_DT,n_estimators=50,random_state=25)

In [70]:
abcl_DT.fit(x_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                         class_weight=None,
                                                         criterion='entropy',
                                                         max_depth=19,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=10,
                                                         min_samples_split=10,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort='deprecated',
                    

In [0]:
y_testpred = abcl_DT.predict(x_test)

In [72]:
print('Training Accuracy for AdaBoost DT: ', accuracy_score(y_train,abcl_DT.predict(x_train)))

Training Accuracy for AdaBoost DT:  0.9999926237005419


In [73]:
print('Training Accuracy for AdaBoost DT: ', accuracy_score(y_test,y_testpred))

Training Accuracy for AdaBoost DT:  0.9437821277767579


In [74]:
print(classification_report(y_test, y_testpred))

              precision    recall  f1-score   support

           1       0.95      0.93      0.94     63514
           2       0.94      0.96      0.95     85045
           3       0.94      0.95      0.94     10747
           4       0.95      0.88      0.91     14998

    accuracy                           0.94    174304
   macro avg       0.95      0.93      0.94    174304
weighted avg       0.94      0.94      0.94    174304



In [75]:
print(confusion_matrix(y_test,y_testpred))

[[59279  4132     1   102]
 [ 2816 81840   169   220]
 [    3   194 10183   367]
 [  463   880   452 13203]]


AdaBoost RF

In [0]:
abcl_RF = AdaBoostClassifier(base_estimator=base_RF,n_estimators=50,random_state=25)

In [16]:
abcl_RF.fit(x_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=RandomForestClassifier(bootstrap=True,
                                                         ccp_alpha=0.0,
                                                         class_weight=None,
                                                         criterion='entropy',
                                                         max_depth=19,
                                                         max_features='auto',
                                                         max_leaf_nodes=None,
                                                         max_samples=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=10,
                                                         min_samples_split=10,
                                    

In [0]:
y_testpred = abcl_RF.predict(x_test)

In [18]:
print('Training Accuracy for AdaBoost RF: ', accuracy_score(y_train,abcl_RF.predict(x_train)))

Training Accuracy for AdaBoost RF:  0.8897513695329328


In [19]:
print('Training Accuracy for AdaBoost RF: ', accuracy_score(y_test,y_testpred))

Training Accuracy for AdaBoost RF:  0.8489937121351202


In [20]:
print(classification_report(y_test, y_testpred))

              precision    recall  f1-score   support

           1       0.87      0.77      0.82     63514
           2       0.83      0.94      0.88     85045
           3       0.83      0.91      0.87     10747
           4       0.91      0.66      0.76     14998

    accuracy                           0.85    174304
   macro avg       0.86      0.82      0.83    174304
weighted avg       0.85      0.85      0.85    174304



In [21]:
print(confusion_matrix(y_test,y_testpred))

[[48613 14822     0    79]
 [ 4779 79757   279   230]
 [    2   280  9780   685]
 [ 2166  1340  1659  9833]]


Need to run from here

GradientBoost

In [0]:
from sklearn.ensemble import GradientBoostingClassifier

In [14]:
gbcl = GradientBoostingClassifier(random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10,
                                  n_estimators = 50)
gbcl.fit(x_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=19,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=10, min_samples_split=10,
                           min_weight_fraction_leaf=0.0, n_estimators=50,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=25, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [0]:
y_testpred = gbcl.predict(x_test)

In [16]:
print('Training Accuracy for GradientBoost Classifier: ', accuracy_score(y_train, gbcl.predict(x_train)))

Training Accuracy for GradientBoost Classifier:  1.0


In [17]:
print('Training Accuracy for GradientBoost Classifier: ', accuracy_score(y_test,y_testpred))

Training Accuracy for GradientBoost Classifier:  0.9713316963466128


In [18]:
print(classification_report(y_test, y_testpred))

              precision    recall  f1-score   support

           1       0.97      0.97      0.97     63514
           2       0.97      0.98      0.98     85045
           3       0.97      0.97      0.97     10747
           4       0.96      0.94      0.95     14998

    accuracy                           0.97    174304
   macro avg       0.97      0.96      0.97    174304
weighted avg       0.97      0.97      0.97    174304



In [19]:
print(confusion_matrix(y_test,y_testpred))

[[61502  1879     0   133]
 [ 1342 83389   100   214]
 [    1   128 10386   232]
 [  243   454   271 14030]]
